In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a chatbot having a conversation with a human."
        ),  # The persistent system prompt
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  # Where the human input will injected
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

llm = ChatOpenAI(model='gpt-4')

# llm = OpenAI(model='gpt-4-vision-preview')

chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

# chat_llm_chain.predict(human_input="my name is angad")

In [6]:
# from langchain.schema.messages import HumanMessage, SystemMessage

# chat = ChatOpenAI(model="gpt-4-vision-preview")
# chat.invoke(
#     [
#         HumanMessage(
#             content=[
#                 {"type": "text", "text": """I'm giving you a floor plan. Describe it.
#                  """},
#                 {
#                     "type": "image_url",
#                     "image_url": {
#                         "url": "https://www.grundriss-schmiede.de/images/buerogrundriss/buerogrundriss.png",
#                         "detail": "auto",
#                     },
#                 },
#             ]
#         )
#     ]
# )

In [2]:
q_string = """Imagine x-y cartesian plane. Point A is at (100,100) and Point B is at (500,200).
I wanna go from point A to B with the following rules:
1. 0<x<600 for any point (x,y) during the journey
2. 0<y<300 for any point (x,y) during the journey
3. There is an obstruction going from (200,0) to (200,200). So i can't go through this obstruction.

Keeping these rules in mind, tell me the distance I need to travel to go from point A to point B""" 

large_string = chat_llm_chain.predict(human_input=q_string)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: Imagine x-y cartesian plane. Point A is at (100,100) and Point B is at (500,200).
I wanna go from point A to B with the following rules:
1. 0<x<600 for any point (x,y) during the journey
2. 0<y<300 for any point (x,y) during the journey
3. There is an obstruction going from (200,0) to (200,200). So i can't go through this obstruction.

Keeping these rules in mind, tell me the distance I need to travel to go from point A to point B

> Finished chain.


In [4]:
# Split the string using the newline character
lines = large_string.split('\n')

# Print each line
for line in lines:
    print(line)

To go from Point A to Point B while avoiding the obstruction, you would need to move up or down before you can go straight to the right (or east, if you are thinking in terms of a map). Since the obstruction is from (200,0) to (200,200), you could go up to (100,300), then to (200,300), then down to (200,200), and finally to (500,200).

Here's how the distance would break down:

1. From A (100,100) to (100,300): This is a vertical move of 200 units.
2. From (100,300) to (200,300): This is a horizontal move of 100 units.
3. From (200,300) to B (500,200): This is a diagonal move. You can calculate the distance using the Pythagorean theorem, which is √[(500-200)² + (300-200)²] = √[300² + 100²] = √(100000) = 316.23 units (rounded to 2 decimal places).

Adding it all up, the total distance you would need to travel is 200 + 100 + 316.23 = 616.23 units.


In [23]:
q_string = """u are walking through the second wall""" 
large_string = chat_llm_chain.predict(human_input=q_string)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: Imagine x-y cartesian plane. Point A is at (100,100) and Point B is at (500,200).
I wanna go from point A to B with the following rules:
1. 0<x<600 for any point (x,y) during the journey
2. 0<y<300 for any point (x,y) during the journey
3. There is an obstruction going from (200,0) to (200,200). So i can't go through this obstruction.
4. There is another obstruction going from (400,0) to (400,300). So i can't go through this obstruction



There is a guideline that the distance from point A to point B should be less than 45. Does this configuration 
follow the guideline? Please keep the rules 1 to 4 in mind.
AI: Given the obstructions and constraints, the shortest path would be to travel from A to the point just before the first obstruction (200,100), then move straight up to the point just above the obstruction (200,200), then move right to the point just be

In [9]:

# Split the string using the newline character
lines = large_string.split('\n')

# Print each line
for line in lines:
    print(line)

I apologize for the confusion. You're absolutely correct. Let's correct that.

1. The person starts at the first staircase at (100, 100).

2. The person will need to walk to the end of the first wall at (200, 200). This distance is 100 units in the y direction.

3. Then, the person must walk along the x-axis to the start of the second wall at (400, 200). This distance is 200 units in the x direction.

4. At this point, the person cannot walk straight to the staircase because there's a wall from (400, 100) to (400, 300). They must first walk upwards along the y-axis to the end of this wall at (400, 300). This distance is 100 units in the y direction.

5. Finally, the person will walk from (400, 300) to the second staircase at (500, 200). This involves a walk of 100 units in the x direction and 100 units in the -y direction.

So, the total shortest distance the person would need to walk is: 100 (to first wall) + 200 (to second wall along x-axis) + 100 (to end of second wall) + 100 (to se

In [13]:
q_string = """bro you used it again despite me telling you its not a method""" 
large_string = chat_llm_chain.predict(human_input=q_string)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: I have many floors and walls at different levels in a Revit project. For a given floor, i wanna check if it has a terrace.
A floor will have terrace if it has any region that is not bounded by walls from all sides. so first i wanna check which walls intersect with
the floor. Then i wanna check if these walls completely surrounf the floor from all sides. use uidoc.selection where uidoc=__revit__.activeuidocument
AI: To achieve this, you need to use the Revit API, which allows you to access and manipulate the underlying data of your Revit model. Here is a simplified version of how you might approach this:

First, you need to get all the floors and walls in your project:

```python
# Get all the floors in the project
floors = FilteredElementCollector(doc).OfCategory(BuiltInCategory.OST_Floors).WhereElementIsNotElementType().ToElements()

# Get all the walls in t

In [14]:

# Split the string using the newline character
lines = large_string.split('\n')

# Print each line
for line in lines:
    print(line)

I apologize for the confusion. You are right, the `get_Geometry()` method doesn't exist. The proper way to get an element's geometry in Revit API is using the `Geometry` property. 

Here's a corrected version of the previous example:

```python
# Get all the floors in the project
floors = FilteredElementCollector(doc).OfCategory(BuiltInCategory.OST_Floors).WhereElementIsNotElementType().ToElements()

# Get all the walls in the project
walls = FilteredElementCollector(doc).OfCategory(BuiltInCategory.OST_Walls).WhereElementIsNotElementType().ToElements()

options = Options()

for floor in floors:
    floor_geom = floor.Geometry[options]

    for floor_solid in floor_geom:
        if isinstance(floor_solid, Solid): # Checking if the geometry instance is a Solid
            for wall in walls:
                wall_geom = wall.Geometry[options]

                for wall_solid in wall_geom:
                    if isinstance(wall_solid, Solid): # Checking if the geometry instance is a Solid
  